# Bayesian optimization
This file implements our bayesian optimization method. Run this to find the best parameters, and then use them in the run.ipynb!

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)


Mounted at /content/drive


In [2]:
## Folder path to Google Drive project directory
victor = True
tomas = False
edwin = False

if tomas:
  path_to_your_folder =  "/content/drive/MyDrive/ML_google_colab/Project 2/submission_branch/ml-project-2-roadmen-bruv"
elif victor:
  path_to_your_folder = "/content/drive/MyDrive/EPFL/MachineLearningMA3/ml-sub/ml-project-2-roadmen-bruv"
elif edwin:
  path_to_your_folder = "/content/drive/MyDrive/ml-project-2-roadmen-bruv"


%cd $path_to_your_folder

/content/drive/MyDrive/EPFL/MachineLearningMA3/ml-sub/ml-project-2-roadmen-bruv


## Libraries

In [3]:
from IPython.display import clear_output
!pip install git+https://github.com/qubvel/segmentation_models.pytorch --quiet
!pip install -U albumentations --quiet
!pip install bayesian-optimization --quiet
clear_output()

In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import segmentation_models_pytorch as smp
from segmentation_models_pytorch import utils as smp_utils
import albumentations as albu

import sys
sys.path.append("./utils")
sys.path.append("./helpers")

import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import torch

from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

import pandas as pd

## Utils
from dataset import Dataset, get_preprocessing

## Helpers
from mask_to_submission import masks_to_submission

## Custom F1 score
from f1scorepatch import F1ScorePatch

## Bayesian optimization library
from bayes_opt import BayesianOptimization

## To empty cache
import gc ###
gc.collect()
torch.cuda.empty_cache()

## Command board
**NB:** We run a dummy instance to show that our framework works, which explains the low number of epochs. The results showed in the report were trained using 40 epochs and data augmentation.

In [49]:
PARAMS = {
  'MODELS' : ["Unet"], # Available : "Unet","DeepLabV3","FPN", "UnetPlusPlus"
  'ENCODER' : 'resnet34',
  'ENCODER_WEIGHTS' : 'imagenet',
  'NB_EPOCHS' : 5
  ,
  'ACTIVATION' : 'sigmoid', # could be None for logits or 'softmax2d' for multiclass segmentation,
  'DATA_AUGMENTATION' : False, #choose whether to train with augmented dataset,
  'LOSS_TYPE': "dice", # Possible: ["dice", "tversky", "custom"]
  'METRIC_TYPE': "fscore", # Possible: ["custom", "fscore"]
  'CLASSES' : ['road'],
}

## Data importation

In [50]:
## Importing preprocessing fonction
preprocessing_fn = smp.encoders.get_preprocessing_fn(PARAMS["ENCODER"], PARAMS["ENCODER_WEIGHTS"])

#Defining folder paths
PATH_TR_IMG_AUG_RAW = "./data/data_train_augmented/raw/images/"
PATH_TR_MASK_AUG_RAW = "./data/data_train_augmented/raw/masks/"
PATH_VAL_IMG_RAW = "./data/data_validation/raw/images/"
PATH_VAL_MASK_RAW = "./data/data_validation/raw/masks/"
PATH_TR_IMG_AUG = "./data/data_train_augmented/images/"
PATH_TR_MASK_AUG = "./data/data_train_augmented/masks/"
PATH_VAL_IMG = "./data/data_validation/images/"
PATH_VAL_MASK = "./data/data_validation/masks/"


#change paths for the training and validation datasets depending on wether we want data augmentation or not
if PARAMS["DATA_AUGMENTATION"]:
  training_path_img = PATH_TR_IMG_AUG
  training_path_mask = PATH_TR_MASK_AUG
  validation_path_img = PATH_VAL_IMG
  validation_path_mask = PATH_VAL_MASK
else:
  training_path_img = PATH_TR_IMG_AUG_RAW
  training_path_mask = PATH_TR_MASK_AUG_RAW
  validation_path_img = PATH_VAL_IMG_RAW
  validation_path_mask = PATH_VAL_MASK_RAW

#create training and validation datasets
train_dataset = Dataset(
    training_path_img,
    training_path_mask,
    preprocessing=get_preprocessing(preprocessing_fn),
    classes=["road"])


valid_dataset = Dataset(
    validation_path_img,
    validation_path_mask,
    preprocessing=get_preprocessing(preprocessing_fn),
    classes=["road"],
)

#create the loaders for both datasets
train_loader = DataLoader(train_dataset, batch_size=30, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=20, shuffle=False)

## Bayesian optimization

### Training function

This is the exact same training process that can be found in run.ipynb, but adapted into a function to allow interaction with the [BayesianOptimization library](https://github.com/bayesian-optimization/BayesianOptimization).

In [51]:
def model_training(_alpha=0.5, _beta=0.5, _gamma=0.75, _lr=1e-4, print_logs=False):
    # Ensure that the model and optimizer are reinitialized
    best_validation_scores = []
    models = None
    optimizer = None
    # Instantiating a new model for each iteration
    models = [[smp.create_model(model_name, encoder_name=PARAMS["ENCODER"], encoder_weights=PARAMS["ENCODER_WEIGHTS"], in_channels=3, classes=1), model_name] for model_name in PARAMS["MODELS"]]

    # Defining the loss
    loss_fn = smp.losses.tversky.TverskyLoss(mode='binary', alpha=_alpha, beta=_beta, gamma=_gamma)
    loss_fn.__name__ = 'Tversky_Loss'
    loss_name = 'Tversky_Loss'

    # Defining the metric
    metrics_training = [smp_utils.metrics.Fscore()]
    metrics_validation = [smp_utils.metrics.Fscore(), F1ScorePatch(activation='sigmoid')]
    metric_name_val = "f1_score_patch" # can also pick "fscore"

    # Train models for NB_EPOCHS
    for model, model_name in models:
        optimizer = torch.optim.Adam([
            dict(params=model.parameters(), lr=_lr),
        ])

        train_epoch = smp.utils.train.TrainEpoch(
            model,
            loss=loss_fn,
            metrics=metrics_training,
            optimizer=optimizer,
            device="cuda",
            verbose=print_logs,
        )

        valid_epoch = smp.utils.train.ValidEpoch(
            model,
            loss=loss_fn,
            metrics=metrics_validation,
            device="cuda",
            verbose=print_logs,
        )

        max_score = 0
        train_loss_array = []
        validation_loss_array = []
        validation_fscore_array = []

        scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, verbose=print_logs)

        for i in range(0, PARAMS["NB_EPOCHS"]):
            if print_logs:
                print('\nEpoch: {}'.format(i))
            train_logs = train_epoch.run(train_loader)
            valid_logs = valid_epoch.run(valid_loader)

            train_loss_array.append(train_logs[loss_name])
            validation_loss_array.append(valid_logs[loss_name])
            validation_fscore_array.append(valid_logs[metric_name_val])

            scheduler.step(valid_logs[metric_name_val])

            # Only saving the model if the validation metric increases
            if max_score < valid_logs[metric_name_val]:
                max_score = valid_logs[metric_name_val]
                if print_logs:
                    print('New iteration best is found!')

        best_validation_scores.append(max_score)
        if print_logs:
            print(best_validation_scores)
            print(max(best_validation_scores))

        # Ensure that the model is deleted from the GPU
        del model, optimizer
        torch.cuda.empty_cache()
        gc.collect()

    return max(best_validation_scores)


### BO function

In [52]:
def bayesian_optimization(preloaded_points = []):
    """
    Perform Bayesian optimization to find the best set of parameters for model training.

    Args:
        preloaded_points (list, optional): List of preloaded points to initialize the optimization process.
                                           Each point should be a dictionary with 'params' and 'target' keys.

    Returns:
        None
    """

    # Define the bounds for each parameter
    pbounds = {
        '_alpha': (0.3, 0.7),  # Range for alpha
        '_beta': (0.3, 0.7),   # Range for beta
        '_gamma': (0.5, 2),    # Range for gamma
        '_lr': (1e-5, 1e-2)    # Range for learning rate
    }

    optimizer = BayesianOptimization(
        f=model_training,
        pbounds=pbounds,
        random_state=1,
    )

    # Register preloaded points
    for point in preloaded_points:
        optimizer.register(params=point['params'], target=point['target'])

    # Perform optimization
    optimizer.maximize(
        init_points=2,  # Number of random initial points
        n_iter=2,      # Number of iterations
    )

    # Print the best parameters found
    print("Best Parameters: ", optimizer.max['params'])

### Optimization
**NB:** The optimization results shown in the report are not displayed here as they were very long to run. Instead we run a dummy instance to show that our framework is functionnal.

There is also a possibilty to import previous results if ever we want to add additional iterations to improve our optimum.

In [53]:
bayesian_optimization()

|   iter    |  target   |  _alpha   |   _beta   |  _gamma   |    _lr    |
-------------------------------------------------------------------------
| 1         | 0.4305    | 0.4668    | 0.5881    | 0.5002    | 0.00303   |
| 2         | 0.539     | 0.3587    | 0.3369    | 0.7794    | 0.003462  |
| 3         | 0.4315    | 0.3031    | 0.3647    | 1.564     | 0.00765   |
| 4         | 0.0       | 0.477     | 0.4654    | 1.322     | 0.007445  |
Best Parameters:  {'_alpha': 0.3587023563268452, '_beta': 0.3369354379075191, '_gamma': 0.7793903170665064, '_lr': 0.0034621516631600474}
